In [ ]:
# default_exp tutorials.sales

# tutorials.sales

> Sample tutorial for how dword can be used to create customized sales videos

In [ ]:
!python3 --version

Python 3.8.3


In [ ]:
from dword.core import *
from dword.utils import *
import pandas as pd
from pathlib import Path
import os

from datetime import date
import calendar
import time

In [ ]:
#hide
# API_KEY = os.environ['API_KEY']
# SECRET_KEY = os.environ['SECRET_KEY']
# from config import *
API_KEY = '1T0B9S9-1ZMMKHG-PP7KRPN-RT3HVVW'
SECRET_KEY = '0e80b4e5-0fe9-49c6-b58f-3c5ac6871def'

Let's start by logging into our DeepWord accounts

In [ ]:
acc = DeepWord(API_KEY, SECRET_KEY)

login successful


It's always a good practice to make sure you have enough credits before proceeding

In [ ]:
acc.available_credits

1143

We will start by importing our csv.

> Note: Here our csv was encoded in a different way, you won't have to pass that parameter most of the time

In [ ]:
df = pd.read_csv('100-contacts.csv', encoding = "ISO-8859-1")
df.head()

,first_name,last_name,company_name,industry,Job Title,address,city,county,state,zip,phone1,phone,email
0,James,Butt,Benton,Marketing,Content strategist,6649 N Blue Gum St,New Orleans,Orleans,LA,70116,504-621-8927,504-845-1427,jbutt@gmail.com
1,Josephine,Darakjy,Chanay,Sales,Compensation analyst,4 B Blue Ridge Blvd,Brighton,Livingston,MI,48116,810-292-9388,810-374-9840,josephine_darakjy@darakjy.org
2,Art,Venere,Chemel,Sales,Investment advisor,8 W Cerritos Ave #54,Bridgeport,Gloucester,NJ,8014,856-636-8749,856-264-4130,art@venere.org
3,Lenna,Paprocki,Feltz Printing Service,Marketing,Marketing technologist,639 Main St,Anchorage,Anchorage,AK,99501,907-385-4412,907-921-2010,lpaprocki@hotmail.com
4,Donette,Foller,Printing Dimensions,Medical,Full stack developer,34 Center St,Hamilton,Butler,OH,45011,513-570-1893,513-549-4561,donette.foller@cox.net


We want to use the columns of this csv to create custom messages for users. Let's write some functions for the same. The first function we will write is to get the current day. For this we will use the date and calendar libraries

In [ ]:
def get_day():
    my_date = date.today().weekday()
    return calendar.day_name[my_date]

In [ ]:
get_day()

'Monday'

Next we'll write a function to greet the users. For this we'll use 4 columns from the csv

In [ ]:
def get_text(first_name, job_title, industry, company_name):
    return (f"Hello {first_name}, hope your {get_day()} morning is going well."
            f" We know that {job_title}'s like you in the {industry} industry are always looking"
            f" to better connect with your customers and employees. My name is Carl from Deep Word"
            f" and we enable companies like {company_name} to generate individual personalized videos"
            f" for their customers and employees, at scale, with any piece of data you have on these"
            f" individuals. Let me quickly show you how you can create videos, just like this one"
            f" using Deep Word."
           )

In [ ]:
get_text('James', 'Content strategist', 'Marketing', 'Benton')

"Hello James, hope your Monday morning is going well. We know that Content strategist's like you in the Marketing industry are always looking to better connect with your customers and employees. My name is Carl from Deep Word and we enable companies like Benton to generate individual personalized videos for their customers and employees, at scale, with any piece of data you have on these individuals. Let me quickly show you how you can create videos, just like this one using Deep Word."

Great! Now to create our synthetic videos we need two things

1. Video of a person talking
2. Audio we want them to say

For the video, we will use one of DeepWord's video actors. To download them use:

In [ ]:
acc.download_video_actors()

'Successfully downloaded all video actors'

In [ ]:
!ls video_actors/

Anna.mp4    Dalton.mp4  Isaac.mp4   Karen.mp4   Mia.mp4     Richard.mp4
Berto.mp4   Emily.mp4   James.mp4   Marcus.mp4  Micheal.mp4 Sam.mp4
Carlos.mp4  Henry.mp4   Julia.mp4   Mary.mp4    Noelle.mp4  Trey.mp4


In [ ]:
display_video('video_actors/Berto.mp4')

For the audio, we will iterate over the csv, use that row to generate custom text for each user and then use that custom text to generate an audio using text2speech. Here's what that looks like

> Tip: For detailed explanation of text2speech, refer to our [core module tutorial](https://deep-word.github.io/dword/core.html).

In [ ]:
acc._available_languages[:10]

['arabic',
 'bengali',
 'chinese',
 'czech',
 'danish',
 'dutch',
 'english_aus',
 'english_ind',
 'english_uk',
 'english_us']

In [ ]:
lang = "english_uk"
speaker = acc._available_speakers(lang)[1]

In [ ]:
text = get_text('James', 'Content strategist', 'Marketing', 'Benton')

In [ ]:
audio = acc.text2speech(text, lang, speaker)

Successfully generated audio file text2speech.mp3


In [ ]:
display_audio(audio)

Sounds great!! Doesn't it? We can now put it all together

In [ ]:
# video input
video = 'video_actors/Berto.mp4'

# audio input
lang = "english_uk"
speaker = acc._available_speakers(lang)[1]

# iterate over the data frame
for idx, row in df.iterrows():
    
    # generate custom text for each user
    text = get_text(row['first_name'], row['Job Title'], row['industry'], row['company_name'])
    
    # generate custom audio for each user
    audio = acc.text2speech(text, lang, speaker, outfile = f'{idx}_audio.mp3')
    
    acc.generate_video("video_actors/Berto.mp4", audio, title = f"video_{idx}.mp4")
    
    if idx == 5: break

Successfully generated audio file 0_audio.mp3
Successfully generated audio file 1_audio.mp3
Successfully generated audio file 2_audio.mp3
Successfully generated audio file 3_audio.mp3
Successfully generated audio file 4_audio.mp3
Successfully generated audio file 5_audio.mp3


> Tip: You can view the outputs or status of your videos on the DeepWord website as well

Once you have submitted your videos, it will take 10-15 mins per video to generate the output. You can check the status of each video by using list_videos(). Let's check the metadata for the last video

In [ ]:
all_videos = acc.list_videos()
len(all_videos)

941